# Setup

Clone the repository.

In [ ]:
!git clone https://github.com/georgao35/GANSketching_ANN.git GANSketching/

Install requirements.

In [ ]:
!pip install -r GANSketching/requirements.txt

Download pretrained weights.

In [ ]:
!bash weights/download_weights.sh

# Example Commands

## Generate samples from a customized model

### Standing Cat

In [ ]:
!cd GANSketching/ && python generate.py --ckpt weights/photosketch_standing_cat_noaug.pth --save_dir output/samples_standing_cat

### Teaser Cat

In [ ]:
!cd GANSketching/ && python generate.py --ckpt weights/by_author_cat_aug.pth --save_dir output/samples_teaser_cat

## Latent space edits by GANSpace

### add fur to the standing cats

In [ ]:
!cd GANSketching/ && python ganspace.py --obj cat --comp_id 27 --scalar 50 --layers 2,4 --ckpt weights/photosketch_standing_cat_noaug.pth --save_dir output/ganspace_fur_standing_cat

###  close the eyes of the standing cats

In [ ]:
!cd GANSketching/ && python ganspace.py --obj cat --comp_id 45 --scalar 60 --layers 5,7 --ckpt weights/photosketch_standing_cat_noaug.pth --save_dir output/ganspace_eye_standing_cat

## Model Training

Training and evaluating on model trained on PhotoSketch inputs requires running [the Precision and Recall metric](https://github.com/kynkaat/improved-precision-and-recall-metric). The following command pulls the submodule of the forked Precision and Recall [repo](https://github.com/PeterWang512/precision_recall).

In [ ]:
!git submodule update --init --recursive

### Download Datasets and Pre-trained Models

The following scripts downloads our sketch data, our evaluation set, [LSUN](https://dl.yf.io/lsun), and pre-trained models from [StyleGAN2](https://github.com/NVlabs/stylegan2) and [PhotoSketch](https://github.com/mtli/PhotoSketch).

In [ ]:
%%bash
cd GANSketching/

# Download the sketches
bash data/download_sketch_data.sh

# Download evaluation set
bash data/download_eval_data.sh

# Download pretrained models from StyleGAN2 and PhotoSketch
bash pretrained/download_pretrained_models.sh

# Download LSUN cat, horse, and church dataset
bash data/download_lsun.sh

To train FFHQ models with image regularization, please download the [FFHQ dataset](https://github.com/NVlabs/ffhq-dataset) using this [link](https://drive.google.com/file/d/1WvlAIvuochQn_L_f9p3OdFdTiSLlnnhv/view?usp=sharing). This is the zip file of 70,000 images at 1024x1024 resolution. Unzip the files, , rename the `images1024x1024` folder to `ffhq` and place it in `./data/image/`.

### Training Scripts

The example training configurations are specified using the scripts in `scripts` folder. Use the following commands to launch trainings.

#### Train the "horse riders" model

In [ ]:
!cd GANSketching/ && bash scripts/train_photosketch_horse_riders.sh

#### Train the cat face model in Figure. 1 of the paper.

In [ ]:
!cd GANSketching/ && bash scripts/train_teaser_cat.sh

#### Train on a single quickdraw sketch

In [ ]:
!cd GANSketching/ && bash scripts/train_quickdraw_single_horse0.sh

#### Train on sketches of faces (1024px)

In [ ]:
!cd GANSketching/ && bash scripts/train_authorsketch_ffhq0.sh

The training progress is tracked using `wandb` by default. To disable wandb logging, please add the `--no_wandb` tag to the training script.

### Evaluations

Please make sure the evaluation set and model weights are downloaded before running the evaluation.

Use the following script to evaluate the models, the results will be saved in a csv file specified by the ``--output`` flag. ``--models_list`` should contain a list of tuple of model weight paths and evaluation data. Please see `weights/eval_list` for example.

In [ ]:
!cd GANSketching/ && python run_metrics.py --models_list weights/eval_list --output metric_results.csv